# Image Processing for input&outputs of datasets to DSTT and DIP model

# Compile data of SAR images to excel

To match the buoy data date and time to image index

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from openpyxl import Workbook

# Directory containing the images
image_dir = "./FLORIDA COAST 2021 data"

# Create a new Excel workbook
wb = Workbook()

# Select the active worksheet
ws = wb.active

# Add a header
ws.append(["Image Names"])

# List all files in the directory
files = os.listdir(image_dir)

# Iterate through the files and add image names to the worksheet
for file in files:
    # Check if the file is an image (you might want to add more checks if needed)
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
        ws.append([file])

# Save the workbook
excel_filename = "FLORIDA_2019_FILTERED.xlsx"
wb.save(os.path.join(image_dir, excel_filename))

print(f"Image names have been saved to {excel_filename} in {image_dir}")


Image names have been saved to FLORIDA_2019_FILTERED.xlsx in ./FLORIDA COAST 2021 data


# Renaming to format XXXXX.png

Rename for all annotations and orginal image, DSTT and DIP runs the inpainting for each frame by matching the same image index

In [1]:
import os

folder_path = "./INPAINTED_DIP_2021"  # Replace with the path to your folder
counter = 0

# List all files in the folder
files = os.listdir(folder_path)

# Filter only JPEG files (assuming case-insensitive matching)
jpeg_files = [file for file in files if file.lower().endswith(".png")]

# Sort the list of JPEG files for consistent numbering
jpeg_files.sort()

# Iterate through the JPEG files and rename them
for filename in jpeg_files:
    # Create the new filename with leading zeros
    new_filename = f"{counter:05d}.png"

    # Build the full path for the old and new filenames
    old_filepath = os.path.join(folder_path, filename)
    new_filepath = os.path.join(folder_path, new_filename)

    # Rename the file
    os.rename(old_filepath, new_filepath)

    counter += 1

print("Renaming complete.")

Renaming complete.


# Rotation of Image (-90)

Rotation of SAR images needed to fit into model of DSTT and DIP; Change line 24 degrees to rotate back for wave paramater retrieval

In [2]:
from PIL import Image
import os

# Directory containing your images
input_dir = "./FLORIDA COAST 2021 data"

# Directory to save rotated images
output_dir = "./FLORIDA COAST 2021 data(rotated)"

# Ensure the output directory exists, if not create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all files in the input directory
files = os.listdir(input_dir)

for file in files:
    # Check if the file is an image (you might want to add more checks if needed)
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
        # Open the image file
        img = Image.open(os.path.join(input_dir, file))
        
        # Rotate the image by -90 degrees
        rotated_img = img.transpose(Image.ROTATE_270)
        
        # Save the rotated image to the output directory
        rotated_img.save(os.path.join(output_dir, file))

print("Rotation completed!")


Rotation completed!


# Create Synthetic Vessel SAR Image

In [8]:
from PIL import Image
import os

# Directory containing JPG images
jpg_dir = "./FLORIDA COAST 2021 data(rotated)"

# Directory containing PNG images
png_dir = "./SINGAPORE 2021 ships layer"

# Directory to save combined images
output_dir = "./ORG_2021"

# Ensure the output directory exists, if not create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all JPG files in the JPG directory
jpg_files = os.listdir(jpg_dir)

# List all PNG files in the PNG directory
png_files = os.listdir(png_dir)

# Iterate over JPG files
for jpg_file in jpg_files:
    # Check if the file is a JPG image
    if jpg_file.endswith(".jpg") or jpg_file.endswith(".jpeg"):
        # Open the JPG image
        jpg_img = Image.open(os.path.join(jpg_dir, jpg_file))
        
        # Find corresponding PNG file
        corresponding_png = jpg_file.replace(".jpg", ".png")
        if corresponding_png in png_files:
            # Open the PNG image
            png_img = Image.open(os.path.join(png_dir, corresponding_png))
            
            # Ensure PNG image has the same dimensions as the JPG image
            if jpg_img.size == png_img.size:
                # Create a new image with the JPG image as the background
                combined_img = jpg_img.copy()
                
                # Paste the PNG image onto the JPG image
                combined_img.paste(png_img, (0, 0), png_img)
                
                # Save the combined image
                combined_img.save(os.path.join(output_dir, jpg_file))
            else:
                print(f"Dimensions mismatch for {jpg_file} and {corresponding_png}. Skipping...")
        else:
            print(f"No corresponding PNG found for {jpg_file}. Skipping...")

print("Combining images completed!")


Combining images completed!


# Cropping image for DSTT and DIP

For UAV Images:

DSTT - 1245x692

DIP - 1216x896

For SAR Images:

DSTT - 432x240

DIP - 448x256 (remain)

In [12]:
from PIL import Image
import os

# Define the directory where your JPEG images are located
input_directory = './FLORIDA COAST 2021 data(rotated)'

# Define the directory where you want to save the cropped images
output_directory = './GROUNDTRUTH_DSTT_2021'

# Make sure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all the files in the input directory
input_files = os.listdir(input_directory)

for file_name in input_files:
    if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
        # Open the image
        image = Image.open(os.path.join(input_directory, file_name))

        # Get the dimensions of the image
        width, height = image.size

        # Crop the image
        #cropped_image = image.crop((0, 0, width, height - 208)) #1245x692
        #cropped_image = image.crop((14.5, 0, width-14.5, height-4)) #1216x896
        cropped_image = image.crop((0, 0, width-16, height - 16))  #for 432x240

        # Save the cropped image to the output directory
        cropped_image.save(os.path.join(output_directory, file_name))

        # Close the image
        image.close()

print("Cropping completed. Cropped images saved in the output directory.")


Cropping completed. Cropped images saved in the output directory.


# Convert mask annotations to black/white from red/black for DIP

DSTT uses red as annotation for inpainted area and black as background

DIP uses black as annotation for inpainted area and white as background

In [13]:
from PIL import Image
import numpy as np
import os

def convert_image_colors(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to a NumPy array for easier manipulation
    image_array = np.array(image)

    # Identify black pixels and convert to white
    black_pixels = np.all(image_array == [0, 0, 0], axis=-1)
    image_array[black_pixels] = [255, 255, 255]

    # Identify non-white pixels and convert to black
    non_white_pixels = ~np.all(image_array == [255, 255, 255], axis=-1)
    image_array[non_white_pixels] = [0, 0, 0]

    # Convert the NumPy array back to an image
    modified_image = Image.fromarray(image_array)

    return modified_image

if __name__ == "__main__":
    input_folder_path = "./ANNOTATIONS_DIP_2021"
    output_folder_path = "./ANNOTATIONS_DIP_2021"

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Process all images in the input folder
    for filename in os.listdir(input_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            input_file_path = os.path.join(input_folder_path, filename)

            # Convert the image colors
            modified_image = convert_image_colors(input_file_path)

            # Save the modified image to the output folder
            output_file_path = os.path.join(output_folder_path, f"{filename}")
            modified_image.save(output_file_path)

    print("Conversion complete.")


Conversion complete.


# Extracting .png frames from mp4. (DSTT)

DSTT outputs as mp4., extract the frames back as .png to run analysis

In [ ]:
import cv2
import os

def save_all_frames(video_path, dir_path, basename, ext='jpg'):
    
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
            n += 1
        else:
            return

#file path, output location, label
save_all_frames('checkpoints_dstt_LOCATION_result.mp4', 'output/checkpoints_dstt_LOCATION_result', 'frame')